<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install wandb
!git clone https://github.com/Krankile/npmf.git

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##General setup

In [22]:
%%capture
!cd npmf && git pull

import os
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from operator import itemgetter
import pickle

import numpy as np
import math
from numpy.ma.core import outerproduct
import pandas as pd
from pandas.tseries.offsets import Day, BDay
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

import wandb as wb

import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from npmf.utils.colors import main, main2, main3
from npmf.utils.wandb import get_dataset, put_dataset
from npmf.utils.eikon import column_mapping
from npmf.utils.dataset import TimeDeltaDataset
from npmf.utils.tests import pickle_df
from npmf.utils.dtypes import fundamental_types

In [6]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [8]:
np.random.seed(69)

# Create a Neural network class

In [9]:
class MultivariateNetwork(nn.Module):
    def __init__(self, lag_len, cat_len, out_len, hidden_dim):
        super().__init__()

        self.pre = nn.Sequential(
            nn.Linear(lag_len, hidden_dim),
            nn.ReLU(),
        )

        self.predict = nn.Sequential(
            nn.Linear(hidden_dim + cat_len, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_len),
        )


    def forward(self, lags, cats):

        x = self.pre(lags)
        x = torch.cat((x, cats), dim=1)
        x = self.predict(x)

        return x

# Get some data

In [10]:
stock_df = get_dataset("stock-oil-final:latest", project="master-test")
meta_df = get_dataset("meta-oil-final:latest", project="master-test")
fundamentals_df = get_dataset("fundamentals-oil-final:latest", project="master-test")
macro_df = get_dataset("macro-oil-final:latest", project="master-test")

stock_df = stock_df.drop_duplicates(subset=["ticker", "date"])

wandb: Currently logged in as: ankile (krankile). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact stock-oil-final:latest, 77.63MB. 1 files... Done. 0:0:0


# Define a dataset which can iterate through time 

We want to train our neural network like the person experiences the world. I.e. we have a window of time and look at recent financial reports and macro variables to predict future market capitalization. We want to train multiple epochs over one time window and validation period, in this manner we will not have any "learned future" effect which could occur if one were to have epochs run over the all time windows. 

In [23]:
current_time = pd.to_datetime("2010-03-01")
n_reports = 4
training_window = 240
forecast_window = 20

delta_set = TimeDeltaDataset(current_time, forecast_window, training_window, n_reports, stock_df, fundamentals_df, meta_df, macro_df)

# Run the loop! (Like Odd-Geir Lademo)

![picture](https://drive.google.com/uc?id=1Y55gFQSi4Baovmi0kUQGhbgGOBTI03E7)


In [28]:
def mape_loss(target, y_pred):
    return (y_pred - target).abs() / (target.abs() + 1e-8)

In [33]:
from dataclasses import dataclass
from typing import Callable


@dataclass
class RunParams:
    n_reports: int
    training_w: int
    forecast_w: int
    epochs: int
    loss_fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor]

In [27]:
def train_multivar(model, optimizer, data_train, data_val, device, params: RunParams):
    train_losses = []
    val_losses = []
    it = tqdm(range(params.epochs), disable=True)
    for epoch in it: 
        for run_type, dataloader in {"train": data_train, "val": data_val}.items():
            model.train(run_type == "train")
            
            for (stocks_and_fundamentals, meta, macro), forecast in dataloader:
                
                stocks_and_fundamentals = stocks_and_fundamentals.to(device)
                meta = meta.to(device)
                macro = macro.to(device)
                forecast = forecast.to(device)

                pred = model(stocks_and_fundamentals, meta, macro)
                # TODO: Remember to mask this
                loss = params.loss_fn(forecast, pred)

                if run_type == "train":
                    train_losses.append(loss.item())
                    loss.backward()

                    optimizer.step()
                else:
                    val_losses.append(loss.item())

        it.set_postfix({"train_loss": np.mean(train_losses), "val_loss": np.mean(val_losses)})

    return train_losses, val_losses

In [ ]:
def main():
    
    n_reports = 4
    training_window = 240
    forecast_window = 20

    params = RunParams(
        n_reports=4,
        training_w=240,
        forecast_w=20,
        epochs=100,
        loss_fn=mape_loss,
    )

    model = MultivariateNetwork()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    periods = iter(pd.date_range(start="2000-12-31", end="2018-10-31", freq="M"))
    tra = None
    val = TimeDeltaDataset(next(periods), forecast_window, training_window, n_reports, stock_df, fundamentals_df, meta_df, macro_df)

    for period in tqdm(periods):
        tra = val
        val = TimeDeltaDataset(period, forecast_window, training_window, n_reports, stock_df, fundamentals_df, meta_df, macro_df)

        tra_loader = DataLoader(tra, batch_size=128, shuffle=True, num_workers=-1)
        val_loader = DataLoader(val, batch_size=128, shuffle=False, num_workers=-1)


        train_multivar(model, optimizer, tra_loader, val_loader, device, params)


main()

In [ ]:
def run_multivar(hidden_dim):
    #Training loop params
    forecast_window = 16
    amount_of_time_series = 999
    length_of_time_series = 500
    epochs = 200
    batch_size = 111
    


    #_, TS_signal, one_hot_encoding  = time_series_df(amount_of_time_series,length_of_time_series,periods, horisontal_shift, vertical_shift, forecast_window)

    df_train = TS_signal.iloc[-int(amount_of_time_series/3):,:]
    df_val = TS_signal.iloc[:-int(amount_of_time_series/3),:]

    loss_fn = nn.L1Loss()
    
    model = MultivariateNetwork(length_of_time_series, one_hot_encoding.shape[1], forecast_window, hidden_dim)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    return train_multivar(model, optimizer, loss_fn, df_train, df_val, one_hot_encoding, batch_size, forecast_window, epochs, device)